Nama: Irene Maritsa

NIM: H071191026

Pelajari Chapter 5 dari d2l.ai : Deep Learning Computation

In [1]:
import tensorflow as tf
from tensorflow import keras

# 1. Layer dan Blocks

Layer merupakan arsitektur atau struktur yang berisi kumpulan node yang beroperasi secara dan di waktu yang bersamaan, yang mengambil informasi dari layer sebelumnya dan kemudian meneruskan informasi ke layer berikutnya.

Block atau neural network blocks merupakan konsep di mana satu atau lebih layer digambarkan / diabstraksi menjadi satu komponen (block). Salah satu manfaat utama dari abstraksi blocks adalah blocks ini bisa lagi dikombinasikan menjadi struktur yang lebih besar (biasanya secara rekursif).

Dengan menggunakan konsep blocks ini, kita tetap dapat menulis kode yang ringkas dan masih mengimplementasikan neural net yang kompleks. Untuk mengimplementasikan blocks, kita membuat fungsi atau jika ingin fleksibel, kelas turunan dari tensorflow.keras.Model yang memiliki fungsi forward propagation (fungsi call()) . 

Contoh implementasi kode


In [2]:
# Data: digenerate secara random
X = tf.random.uniform((2, 20))

X

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[0.86675084, 0.29700303, 0.7026851 , 0.48117244, 0.56571317,
        0.42220592, 0.03085005, 0.09053874, 0.5019802 , 0.7673718 ,
        0.62660456, 0.9781785 , 0.5200552 , 0.95356095, 0.2855531 ,
        0.65117395, 0.33992577, 0.58180165, 0.29142535, 0.6895231 ],
       [0.52908766, 0.91594005, 0.15600145, 0.7724452 , 0.61576056,
        0.66444564, 0.5583708 , 0.58931804, 0.5205798 , 0.05308056,
        0.26320124, 0.5118493 , 0.778798  , 0.8806647 , 0.12430382,
        0.5067482 , 0.3468107 , 0.08116746, 0.49459398, 0.47376442]],
      dtype=float32)>

Misalnya model yang akan dijadikan blok merupakan model seperti ini

In [3]:
block1 = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])

Bentuk bloknya

In [4]:
class Block1(keras.Model):
    def __init__(self):
        super().__init__()
        self.layer_list= []
        self.layer_list.append(keras.layers.Dense(256, activation=tf.nn.relu))
        self.layer_list.append(keras.layers.Dense(10))
        self.layer_list.reverse()
    
    def call(self, X):
        result_temp= X

        # Melakukan forward propagation sampai layer terakhir dalam list 
        while bool(self.layer_list):
            result_temp= self.layer_list.pop()(result_temp)

        return result_temp
        # return 1

In [5]:
net= Block1()
net(X) # Sama saja dengan net.call(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 8.01609233e-02, -2.62070447e-01,  1.03592329e-01,
        -4.50591035e-02,  4.39621555e-03,  1.91465467e-01,
         2.75752097e-01, -3.06465566e-01,  4.10254523e-02,
         3.06633890e-01],
       [ 1.09469488e-01, -1.10643461e-01,  8.38672891e-02,
        -2.58534827e-04,  1.59429014e-02,  1.02652401e-01,
         3.26629996e-01, -1.02953464e-01, -8.62429738e-02,
         5.55656254e-02]], dtype=float32)>

Selanjutnya kita akan membuat block ke dua yang tidak mengandung layer satu per satu tapi mengandung model

In [6]:
class Block2(keras.Model):
    def __init__(self):
        super().__init__()
        self.net= keras.Sequential()
        self.net.add(keras.layers.Dense(256, activation=tf.nn.relu))
        self.net.add(keras.layers.Dense(10))
    
    def call(self, X):
        return self.net(X)

Kemudian satukan

In [7]:
net= keras.Sequential()

# Menambahkan layer pertama yang mana merupakan block 2
net.add(Block2())

# Menambahkan block 1
net.add(Block1())

# # Menambahkan satu lagi layer dense biasa
net.add(keras.layers.Dense(10, activation= 'sigmoid'))

# Melakukan forward propagation
net(X)

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[0.5083747 , 0.55362755, 0.5165092 , 0.3991169 , 0.4937354 ,
        0.4360028 , 0.5363945 , 0.5499353 , 0.44155607, 0.60816795],
       [0.53501   , 0.49403536, 0.49773425, 0.45228022, 0.50313896,
        0.45818245, 0.5485296 , 0.5309845 , 0.466479  , 0.5492742 ]],
      dtype=float32)>

# 2. Parameter Management

Parameter management atau manajemen parameter termasuk (Parameter dalam kasus neural network contohnya weight dan bias, fungsi loss atau learning rate merupakan hyperparameter):

Mengakses parameter untuk debugging, diagnostik, dan visualisasi.
Inisialisasi parameter.
Berbagi parameter dengan komponen model lain.



## Mengakses Parameter

Model:

In [8]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu),
    keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[-0.11810564],
       [-0.57233703]], dtype=float32)>

Mengambil weight dan bias dari setiap layer net

In [9]:
net.weights
# Atau net.get_weights()

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.5770101 , -0.8510938 , -0.7193422 ,  0.49523228],
        [ 0.22963458,  0.5008809 ,  0.6824108 , -0.8624928 ],
        [ 0.6638021 ,  0.08449888,  0.8158731 ,  0.37731212],
        [-0.4486405 , -0.67031264, -0.33020413,  0.21575874]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'dense_10/kernel:0' shape=(4, 1) dtype=float32, numpy=
 array([[-0.7512425 ],
        [ 0.36477852],
        [ 0.3069142 ],
        [-0.9918738 ]], dtype=float32)>,
 <tf.Variable 'dense_10/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

Untuk mendapatkan parameter pada layer yang ingin diakses, dilakukan dengan memanggil atribut .layers[index dari layer] dari model sequential. Misal layer yang ingin diakses parameternya yaitu layer indeks ke-1

net.layers[1]

Selanjutnya untuk mengakses parameter weight dan bias

In [10]:
net.layers[1].weights

[<tf.Variable 'dense_9/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.5770101 , -0.8510938 , -0.7193422 ,  0.49523228],
        [ 0.22963458,  0.5008809 ,  0.6824108 , -0.8624928 ],
        [ 0.6638021 ,  0.08449888,  0.8158731 ,  0.37731212],
        [-0.4486405 , -0.67031264, -0.33020413,  0.21575874]],
       dtype=float32)>,
 <tf.Variable 'dense_9/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>]

Outputnya berupa array, indeks ke-0 merupakan weights, indeks ke-1 merupakan bias

Untuk layer yang berupa block (memiliki layer lagi di dalamnya), kita tetap bisa mengakses nya seperti ini:

net.layers[0].layers[1].layers[1].weights[1]

### Inisialisasi Parameter

Secara default (untuk tensorflow), matriks weight diinisialisasikan berdasarkan range yang dikomputasi dari dimensi input dan output, sementara parameter bias mempunyai nilai default 0.

Misal menginisialisasi weight sebagai variabel random Gaussian dengan standar deviasi 0.01, dan menginisialisasi parameter bias dengan nilai 0.

In [11]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[1].weights[1]

(<tf.Variable 'dense_11/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[-0.02091935,  0.01042824, -0.02560832,  0.00581529],
        [-0.01222676, -0.00815193, -0.00367557, -0.01373396],
        [-0.01474115,  0.01481128, -0.00560522,  0.00126118],
        [ 0.00658395,  0.00148005, -0.02134722, -0.01984916]],
       dtype=float32)>,
 <tf.Variable 'dense_11/bias:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>)

Kita juga bisa menginisialisasikan weight yang berbeda pada setiap layer, misal weight layer dense kedua diisi dengan angka konstan 0.5.

In [12]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(4, activation=tf.nn.relu, 
                       kernel_initializer=keras.initializers.Constant(0.5), 
                       bias_initializer=tf.zeros_initializer()),
    keras.layers.Dense(1)
])

net(X)
net.layers[1].weights[0], net.layers[2].weights[0]

(<tf.Variable 'dense_13/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[ 0.00210222, -0.02335702, -0.00528331,  0.00504105],
        [ 0.00683558,  0.01486932,  0.00776506,  0.00148767],
        [ 0.01109169, -0.0249704 ,  0.00198765, -0.00016486],
        [ 0.00205768,  0.02211519, -0.0042257 ,  0.015715  ]],
       dtype=float32)>,
 <tf.Variable 'dense_14/kernel:0' shape=(4, 4) dtype=float32, numpy=
 array([[0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5],
        [0.5, 0.5, 0.5, 0.5]], dtype=float32)>)

Kita juga bisa membuat subclass kita sendiri untuk inisialisasi weight jika cara di atas kurang flexibel, caranya yaitu kita membuat class subclass dari keras.initializers.Initializer , lalu mengimplementasikan method call(shape, dtype) yang me-return tensor (berisi weight inisial) yang isinya sesuai kemauan kita

In [13]:
# Di sini kita membuat class untuk inisialisasi weight
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

In [14]:
MyInit().__call__(shape=(3, 3), dtype=tf.float32)

<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[-0.      , -8.486891,  0.      ],
       [-8.794742, -0.      , -5.529537],
       [-9.116373,  8.338263, -0.      ]], dtype=float32)>

In [15]:
net = keras.models.Sequential([
    keras.layers.Flatten(),
    # Memasukkan kelas initializer pada parameter kernel_initializer
    keras.layers.Dense(4, activation=tf.nn.relu, kernel_initializer=MyInit()),
    keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<tf.Variable 'dense_16/kernel:0' shape=(4, 4) dtype=float32, numpy=
array([[ 6.7792206, -6.067848 ,  0.       , -6.0576963],
       [ 0.       , -0.       ,  9.891411 , -0.       ],
       [-8.1249   , -9.604306 , -5.5695224, -0.       ],
       [-8.809593 , -8.938206 , -5.055747 , -9.341871 ]], dtype=float32)>


# 3. Deferred Initialization
Inisialisasi weight pada model ditangguhkan sebelum data di-pass ke model. Ini dikarenakan model masih belum mengetahui input shape dari data, maka dari itu model menangguhkan inisialisasi parameter, konsep ini juga membuat kita tidak perlu mendefinisikan input shape dari setiap layer, yang mana pastinya sangat membantu terutama jika dataset berupa gambar (yang mana mempunyai shape yang dinamis)

Contohnya seperti kasus berikut, pertama-tama kita membuat model lalu memanggil weight nya

In [16]:
net = keras.models.Sequential([
    keras.layers.Dense(256, activation=tf.nn.relu),
    keras.layers.Dense(10),
])
[net.layers[i].get_weights() for i in range(len(net.layers))]

[[], []]

Kita bisa melihat weightnya masih kosong karena belum diinisialisasikan sama sekali

In [17]:
# Feed model dengan X
net(X)

[w.shape for w in net.get_weights()], net.layers[0].get_weights()[0]

([(4, 256), (256,), (256, 10), (10,)],
 array([[ 0.06456782, -0.07295229,  0.1219438 , ...,  0.02922186,
          0.05952103,  0.04227482],
        [ 0.10914555,  0.12628713,  0.1056388 , ...,  0.06826791,
         -0.05991349, -0.10691746],
        [ 0.13555983,  0.12912762, -0.06503145, ..., -0.00182062,
          0.13520795, -0.07534514],
        [-0.10921635, -0.13497168, -0.12281566, ...,  0.09267625,
         -0.13642971, -0.12244417]], dtype=float32))

Barulah setelah data di-pass weightnya terinisialisasi. Ini dinamakan Deferred Initialization (penangguhan inisialisasi).

# 4. Custom Layers
Pada kasus-kasus tertentu, kita berhadapan dengan masalah di mana kita membutuhkan layer yang belum pernah dibuat atau belum ada pada library, untuk mengatasi masalah tersebut, kita bisa membuat layer kustom kita sendiri. Membuat layer kustom sama saja seperti membuat blok. Terdapat 2 jenis kustom layer: layer tanpa atau dengan parameter (seperti weight atau bias)

### Layer tanpa parameter


Layer tanpa parameter merupakan layer yang tidak mempunyai weight dan bias di dalamnya (seperti layer flatten). Misal kita ingin membuat layer yang mengurangi input dengan satu

In [18]:
class MinusOneLayer(keras.Model):
    def __init__(self):
        super().__init__()

    def call(self, input):
        return input - tf.ones(input.shape)

In [19]:
layer_wo_params= MinusOneLayer()
layer_wo_params(tf.constant([1.,2.,3.]))

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 1., 2.], dtype=float32)>

Mengimplementasikannya pada model

In [20]:
net = keras.Sequential([keras.layers.Dense(128), MinusOneLayer()])

### Layer dengan parameter
Layer dengan parameter mempunyai dan melibatkan weight dan bias pada operasinya. Misal kita ingin membuat layer dense yang berfungsi sama dengan layer dense pada umumnya (input * weight + bias lalu aktivasi). Walaupun kita mengatur/inisialisasi parameter semau kita, parameter juga tetap berubah saat proses back propagation

In [21]:
class MyDense(keras.Model):
    # Di sini kita berikan dua parameter saja yaitu untuk output dan aktivasi (tentu saja kita bisa menambahkannya sesuai kebutuhan)
    def __init__(self, units, activation=None):
        super().__init__()
        self.units = units
        self.activation= activation

    # Pada method build kita inisialisasi weight dan bias, fungsi build hanya dijalankan satu kali
    # Jadi weight dan bias tetap akan ter-update saat back propagation
    def build(self, X_shape):
        self.weight = self.add_weight(name='weight',
            shape=[X_shape[-1], self.units],
            initializer=tf.random_normal_initializer())
        self.bias = self.add_weight(
            name='bias', shape=[self.units],
            initializer=tf.zeros_initializer())

    # Pada fungsi call kita melakukan operasi (input * weight + bias lalu aktivasi)
    def call(self, X):
        linear = tf.matmul(X, self.weight) + self.bias

        if self.activation is None:
            return linear
        
        return self.activation(linear)

In [22]:
temp_dense= MyDense(5)
temp_dense(X)

temp_dense.get_weights()

[array([[ 0.02790201, -0.02960716,  0.01616521,  0.01119643,  0.12988684],
        [ 0.04065468,  0.02429028,  0.06017286,  0.00451622, -0.079991  ],
        [ 0.02470844, -0.07201657,  0.07547449,  0.07829442,  0.03545597],
        [ 0.00251699,  0.00978419,  0.06741989,  0.09638153, -0.01885505]],
       dtype=float32), array([0., 0., 0., 0., 0.], dtype=float32)]

Selanjutnya membuat model dengan 2 layer MyDense yang telah dibuat

In [23]:
net= keras.Sequential([
    keras.layers.Flatten(),
    MyDense(128, tf.nn.relu), 
    MyDense(64, tf.nn.relu), 
    MyDense(32, tf.nn.relu), 
    MyDense(10, tf.nn.softmax)
])

net.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

Kita akan mengetes model tersebut pada dataset mnist

In [24]:
(x_train, y_train), (x_test, y_test)= keras.datasets.mnist.load_data()
x_train, x_test= x_train/255, x_test/255

11501568/11490434 [==============================] - 0s 0us/step


In [25]:
net.fit(x_train, y_train, epochs=3, batch_size= 128)

Epoch 1/3
469/469 [==============================] - 3s 4ms/step - loss: 0.5339 - accuracy: 0.8415
Epoch 2/3
469/469 [==============================] - 2s 4ms/step - loss: 0.2079 - accuracy: 0.9393
Epoch 3/3
469/469 [==============================] - 2s 4ms/step - loss: 0.1454 - accuracy: 0.9566


In [26]:
net.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.1356 - accuracy: 0.9592


[0.13559050858020782, 0.9592000246047974]

# 5. File I/O

### Save / Load Tensor
Untuk tensor, kita bisa menggunakan numpy.save(namaFile.npy, tensor) untuk menyimpan tensor dan numpy.load(namaFile.npy) untuk memuatnya.

In [43]:
save_x= tf.constant(['a', 'b', 'c'])
save_x

<tf.Tensor: shape=(3,), dtype=string, numpy=array([b'a', b'b', b'c'], dtype=object)>

Menyimpan tensor

In [44]:
import numpy as np

np.save('save-x.npy', save_x)

Meload tensor

In [45]:
temp_x= np.load('save-x.npy', allow_pickle=True)
temp_x

array([b'a', b'b', b'c'], dtype=object)

### Save / Load Model Parameter
Misal model kita seperti ini

In [46]:
class MLP(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.hidden = tf.keras.layers.Dense(units=256, activation=tf.nn.relu)
        self.out = tf.keras.layers.Dense(units=10)

    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.hidden(x)
        return self.out(x)

net= MLP()

Lalu kita melakukan feed forward pada model

In [47]:
Y1= net(X)
Y1

<tf.Tensor: shape=(2, 10), dtype=float32, numpy=
array([[ 0.01608171,  0.09957618, -0.12128264,  0.07072887, -0.04526163,
        -0.07433657,  0.05163891,  0.00424158,  0.03460515,  0.0469794 ],
       [-0.02286452,  0.05513217, -0.05191122,  0.10090058,  0.01621987,
        -0.00874656, -0.03408854,  0.0521233 ,  0.01493379,  0.1082292 ]],
      dtype=float32)>

Sekarang kita akan menyimpan parameter model dengan fungsi save_weights('nama_weight.params')

In [48]:
net.save_weights('mlp.params')

Sekarang kita akan membuat model dengan arsitektur yang sama tapi tidak dilakukan feed forward, sebaliknya parameternya diambil/diload dari file parameter yang telah di-save sebelumnya

In [49]:
net_2= MLP()
net_2.load_weights('mlp.params')

# Feed forward
Y2= net_2(X)

Sekarang kita akan membandingkan variabel Y1 dengan Y2

In [50]:
Y1==Y2

<tf.Tensor: shape=(2, 10), dtype=bool, numpy=
array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True]])>

Hasilnya adalah sama

# 6. GPUs
Mengecek ketersediaan GPU

In [51]:
!nvidia-smi

Sat Apr 23 13:57:30 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P0    75W / 149W |    707MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Kita dapat menentukan device, seperti CPU dan GPU, untuk penyimpanan dan komputasi. Secara default, tensor dibuat di memori utama dan menggunakan CPU untuk komputasi.

In [52]:
tf.device('/CPU:0'), tf.device('/GPU:0'), tf.device('/GPU:1')

(<tensorflow.python.eager.context._EagerDeviceContext at 0x7f215011ec30>,
 <tensorflow.python.eager.context._EagerDeviceContext at 0x7f20df230910>)

Kita bisa menjalankan query untuk mendapatkan berapa banyak GPU dalam sistem

In [53]:
len(tf.config.experimental.list_physical_devices('GPU'))

1

### Tensor dan GPU

Secara default, tensor dibuat dalam CPU, untuk melihat di device mana suatu tensor berada kita bisa menjalankan fungsi seperti ini

In [56]:
x = tf.constant([1, 2, 3])
x.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Outputnya mengatakan GPU karena saya menggunakan google colabs dan telah mengatur runtime pada saat penulisan kode, namun secara default, kalau tidak didefinisikan, akan mengeluarkan output CPU

### Menyetel penggunaan GPU

Misal kita ingin menggunakan GPU pertama atau GPU:0

In [54]:
tf.device('GPU:0')

In [55]:
with tf.device('GPU:0'):
    test = tf.ones((2, 3))

test.device

'/job:localhost/replica:0/task:0/device:GPU:0'

Hati-hati saat pemilihan GPU terutama jika kita memiliki lebih dari satu GPU dalam satu komputer, karena akan terjadi error jika kita melakukan operasi yang melibatkan 2 tensor yang berada di GPU yang berbeda